In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import warnings
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from agents.matchmanager import buildMatchManager
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import RFE


In [2]:
vectors=[]
df=pd.DataFrame()
winners=[]
if __name__ == '__main__':
    for p in ['PlayerDummy','AlphaBetaAgent']:
        for p1 in ['PlayerDummy','AlphaBetaAgent']:
        
            mm = buildMatchManager('', 'scenarioTest1v1', p, p1, seed=42)

            while not mm.end:
                mm.nextStep()
                vectors.append(mm.state.vector())
            old=df.shape[0]
            df=pd.DataFrame(vectors)
            new=df.shape[0]
            for i in range(new-old):
                winners.append(mm.winner)
              
    df['win']=winners

In [3]:
c=df.select_dtypes(include='object').columns

In [4]:
for i in c.array:
    value=df[i].unique()
    le = preprocessing.LabelEncoder()
    le.fit(value)
    df[i]=le.transform(df[i])
    

    

In [ ]:
def random_forest(df):
   
    X = df.drop('win',1)
    y = df['win'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    clf = RandomForestClassifier(n_estimators=100)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    proba = clf.predict_proba(X_test)
    proba = np.around(proba, decimals=2)


    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

    # feature importanti per determinarlo
    feature_imp = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
    print('Feature importance', feature_imp)

    # Creating a bar plot
    sns.barplot(x=feature_imp, y=feature_imp.index)
    # Add labels to your graph
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')
    plt.title("Visualizing Important Features")
    plt.legend()
    plt.show()
    #return X_test, y_test, y_pred, proba


In [5]:
def low_variance(df):
    X = df
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    return sel.fit_transform(X)

In [ ]:
def recur_cross(df):
    X = df.drop('win',1)
    y = df['win'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


    # Create the RFE object and compute a cross-validated score.
    svc = SVC(kernel="linear")
    # The "accuracy" scoring is proportional to the number of correct
    # classifications
    rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
                  scoring='accuracy')
    rfecv.fit(X_train, y_train)
    print("anche qui")

    print("Optimal number of features : %d" % rfecv.n_features_)

    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()

In [6]:
def recur(df):
    X = df.drop('win',1)
    y = df['win'].values

    # Create the RFE object and rank each pixel
    svc = SVC(kernel="linear", C=1)
    rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
    rfe.fit(X, y)
    ranking = rfe.ranking_.reshape(df.shape)

    # Plot pixel ranking
    plt.matshow(ranking, cmap=plt.cm.Blues)
    plt.colorbar()
    plt.title("Ranking of pixels with RFE")
    plt.show()

In [ ]:
def selectfrommodel(df):
    X = df.drop('win',1)
    y = df['win'].values

    feature_names = X.columns
    print(feature_names)
    clf = LassoCV().fit(X, y)
    importance = np.abs(clf.coef_)
    print(importance)
    idx_third = importance.argsort()[-3]
    threshold = importance[idx_third] + 0.01

    idx_features = (-importance).argsort()[:2]
    name_features = np.array(feature_names)[idx_features]
    print('Selected features: {}'.format(name_features))

    sfm = SelectFromModel(clf, threshold=threshold)
    sfm.fit(X, y)
    X_transform = sfm.transform(X)

    n_features = sfm.transform(X).shape[1]
    plt.title(
        "Features from diabets using SelectFromModel with "
        "threshold %0.3f." % sfm.threshold)
    feature1 = X_transform[:, 0]
    feature2 = X_transform[:, 1]
    plt.plot(feature1, feature2, 'r.')
    plt.xlabel("First feature: {}".format(name_features[0]))
    plt.ylabel("Second feature: {}".format(name_features[1]))
    plt.ylim([np.min(feature2), np.max(feature2)])
    plt.show()

# Random Forest

In [ ]:
random_forest(df)

# Remove low variance


In [7]:
var=low_variance(df)

In [ ]:
recur(df)

import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_diabetes
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
diabetes = load_diabetes()

X = diabetes.data
y = diabetes.target

feature_names = diabetes.feature_names
print(feature_names)
clf = LassoCV().fit(X, y)
importance = np.abs(clf.coef_)
print(importance)
idx_third = importance.argsort()[-3]
threshold = importance[idx_third] + 0.01

idx_features = (-importance).argsort()[:2]
name_features = np.array(feature_names)[idx_features]
print('Selected features: {}'.format(name_features))

sfm = SelectFromModel(clf, threshold=threshold)
sfm.fit(X, y)
X_transform = sfm.transform(X)

n_features = sfm.transform(X).shape[1]
plt.title(
    "Features from diabets using SelectFromModel with "
    "threshold %0.3f." % sfm.threshold)
feature1 = X_transform[:, 0]
feature2 = X_transform[:, 1]
plt.plot(feature1, feature2, 'r.')
plt.xlabel("First feature: {}".format(name_features[0]))
plt.ylabel("Second feature: {}".format(name_features[1]))
plt.ylim([np.min(feature2), np.max(feature2)])
plt.show()